Exercise 1

In [ ]:
## Ordinary least squares cannot tell one variable from another when they are perfectly correlated, they can not really analyze outliers either. 

Exercise 2

In [ ]:
## d). All of the above

Exercise 3

In [ ]:
## f). a and b

Exercise 4

In [ ]:
## The model would be suffering from high variance, reduce the regularization parameter.

Exercise 5

In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import Ridge, RidgeCV

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'mckayla-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to read from s3 bucket
file_key = 'CarPrice_Assignment.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##Reading the csv file
car_price = pd.read_csv(file_content_stream)
car_price.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [2]:
## Defining the input and target variables
X = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg']]
Y = car_price['price']

## Defining list to store estimated coefficients at each split
coef = list()

for i in range(0, 1000):
    
    ## Splitting the data into train and test
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2)

    ## Lasso cross-validation to estimate optimal lambda
    lasso_cv = LassoCV(normalize = True, cv = 5).fit(X_train, Y_train)

    ## Building Lasso Regression Model with optimal lambda
    lasso_md = Lasso(alpha = lasso_cv.alpha_, normalize = True).fit(X_train, Y_train)
    
    ## Storing the estimated coefficients
    coef.append(lasso_md.coef_)
    
## Putting the list as a dataframe
coef_df = pd.DataFrame(coef)

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing

In [3]:
coef_df.head()

,0,1,2,3,4,5,6
0,208.214559,111.149014,269.502377,50.322936,2.239219,-37.769834,-5.608476
1,171.005973,108.210778,277.146428,45.543382,1.505601,-114.081280,-0.000000
2,175.204354,113.553611,332.087438,43.965063,2.399821,-110.927274,-0.000000
3,137.837749,110.415520,334.427677,46.175992,2.049132,-110.371404,-0.000000
4,181.021799,101.790703,279.965718,46.336175,1.564357,-114.250159,-10.488402


In [11]:
sum(coef_df[6] == 0.0)

721

In [12]:
## Dropping highwaympg
X = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg']]
Y = car_price['price']

## Splitting the data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [14]:
## Function for l2_normalization
def l2_normalization(X):
    
    x_mean = np.mean(X)
    l2 = np.sqrt(sum(X**2))
    return (X - x_mean) / l2

X_train = X_train.apply(l2_normalization, axis = 1)
X_test = X_test.apply(l2_normalization, axis = 1)

In [15]:
## Linear Regression
lm_md = LinearRegression().fit(X_train, Y_train)

##Predicting on test
lm_pred = lm_md.predict(X_test)

## Compute the MSE of the linear regression model
mse1 = np.mean(np.power(Y_test - lm_pred, 2))
print('The mse of the linear model is', mse1)

The mse of the linear model is 13135364.48017569


In [20]:
## Defining list to store the optimal lambda
op_lamb = list()

for i in range(0, 100):

    ## Ridge cross-validation to estimate optimal lambda
    ridge_cv = RidgeCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100], cv = 5).fit(X_train, Y_train)
    
    ## Extracting the best lambda
    cv_lambda = ridge_cv.alpha_

    ## Building the ridge model
    ridge_md = Ridge(alpha = cv_lambda).fit(X_train, Y_train)
    
    ## Storing the optimal lambda
    op_lamb.append(ridge_md)
    
    ## Predicting on test
    ridge_pred = ridge_md.predict(X_test)

## Computing the mse of the ridge regression model
mse2 = np.mean(np.power(Y_test - ridge_pred, 2))
print('The mse of the ridge model is', mse2)

The mse of the ridge model is 16878155.334958017


In [ ]:
## Based on the data, we can see that the Linear Regression Model has a smaller MSE than the Ridge Regression Model. 
## The Linear Regression Model would be the model to use when predicting car prices.